In [1]:
import numpy as np
import pandas as pd
import polars as pl
import sys
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px


pd.set_option('display.max_columns',None)
import psycopg2


#to scale the data using z-score 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#Algorithms to use
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

#Metrics to evaluate the model
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve

import warnings
warnings.filterwarnings("ignore")

In [27]:
df = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/input.xlsx')
df

,Genes,proteinlocation,Chrom-pos-Ref-Alt
0,ABCG2,p.Q141K,"4-88131171-G-T,"
1,CACNA1S,p.R1086H,1-201061290-TCT-ATG
2,CACNA1S,p.R1086H,1-201061290-TCT-GTG
3,CACNA1S,p.R1086H,1-201060815-C-T
4,CACNA1S,p.R174W,1-201091993-G-A
...,...,...,...
941,UGT1A1,rs3064744,"2-233760234-ATATA-ATATATATATATA,"
942,UGT1A1,rs3064744,"2-233760234-ATATA-ATA,"
943,VKORC1,rs9923231,"16-31096368-C-A,"
944,VKORC1,rs9923231,"16-31096368-C-T,"


In [24]:
data = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/main_rsID_data.xlsx')
data

,genesymbol,proteinlocation
0,CACNA1S,rs1800559
1,CACNA1S,rs772226819
2,DPYD,rs114096998
3,DPYD,rs148799944
4,DPYD,rs140114515
...,...,...
625,CYP2D6,rs72549353
626,CYP2D6,rs72549351
627,CYP3A5,rs41303343
628,CYP2D6,rs5030655


In [25]:
df = pd.merge(df, data, on='proteinlocation', how = 'left', sort=False)
df

,Genes,proteinlocation,Chrom-pos-Ref-Alt,genesymbol
0,ABCG2,p.Q141K,"4-88131171-G-T,",NaN
1,CACNA1S,p.R1086H,1-201061290-TCT-ATG,NaN
2,CACNA1S,p.R1086H,1-201061290-TCT-GTG,NaN
3,CACNA1S,p.R1086H,1-201060815-C-T,NaN
4,CACNA1S,p.R174W,1-201091993-G-A,NaN
...,...,...,...,...
941,rs3064744,rs3064744,"2-233760234-ATATA-ATATATATATATA,",UGT1A1
942,rs3064744,rs3064744,"2-233760234-ATATA-ATA,",UGT1A1
943,rs9923231,rs9923231,"16-31096368-C-A,",VKORC1
944,rs9923231,rs9923231,"16-31096368-C-T,",VKORC1


In [26]:
df.to_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/input.xlsx', index=False)

In [28]:
df['Chrompos-Ref-Alt_new'] = df['Chrom-pos-Ref-Alt'].str.split(',')
df = df.explode('Chrompos-Ref-Alt_new')
df

,Genes,proteinlocation,Chrom-pos-Ref-Alt,Chrompos-Ref-Alt_new
0,ABCG2,p.Q141K,"4-88131171-G-T,",4-88131171-G-T
0,ABCG2,p.Q141K,"4-88131171-G-T,",
1,CACNA1S,p.R1086H,1-201061290-TCT-ATG,1-201061290-TCT-ATG
2,CACNA1S,p.R1086H,1-201061290-TCT-GTG,1-201061290-TCT-GTG
3,CACNA1S,p.R1086H,1-201060815-C-T,1-201060815-C-T
...,...,...,...,...
943,VKORC1,rs9923231,"16-31096368-C-A,",
944,VKORC1,rs9923231,"16-31096368-C-T,",16-31096368-C-T
944,VKORC1,rs9923231,"16-31096368-C-T,",
945,VKORC1,rs9923231,"16-31096368-C-G,",16-31096368-C-G


In [29]:
df.dropna(subset=['Chrompos-Ref-Alt_new'], inplace=True)
df

,Genes,proteinlocation,Chrom-pos-Ref-Alt,Chrompos-Ref-Alt_new
0,ABCG2,p.Q141K,"4-88131171-G-T,",4-88131171-G-T
0,ABCG2,p.Q141K,"4-88131171-G-T,",
1,CACNA1S,p.R1086H,1-201061290-TCT-ATG,1-201061290-TCT-ATG
2,CACNA1S,p.R1086H,1-201061290-TCT-GTG,1-201061290-TCT-GTG
3,CACNA1S,p.R1086H,1-201060815-C-T,1-201060815-C-T
...,...,...,...,...
943,VKORC1,rs9923231,"16-31096368-C-A,",
944,VKORC1,rs9923231,"16-31096368-C-T,",16-31096368-C-T
944,VKORC1,rs9923231,"16-31096368-C-T,",
945,VKORC1,rs9923231,"16-31096368-C-G,",16-31096368-C-G


In [30]:
df.to_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/shyam_split.xlsx', index=False)

In [31]:
df = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/shyam_split.xlsx')
df

,Genes,proteinlocation,Chrompos-Ref-Alt_new
0,ABCG2,p.Q141K,4-88131171-G-T
1,CACNA1S,p.R1086H,1-201061290-TCT-ATG
2,CACNA1S,p.R1086H,1-201061290-TCT-GTG
3,CACNA1S,p.R1086H,1-201060815-C-T
4,CACNA1S,p.R174W,1-201091993-G-A
...,...,...,...
1141,UGT1A1,rs3064744,2-233760234-ATATA-ATATATATATATA
1142,UGT1A1,rs3064744,2-233760234-ATATA-ATA
1143,VKORC1,rs9923231,16-31096368-C-A
1144,VKORC1,rs9923231,16-31096368-C-T


In [77]:
grp1 = df.groupby(['Genes', 'proteinlocation']).agg({'Chrompos-Ref-Alt_new': lambda x: ','.join(str(i) for i in x.unique())}).reset_index()
grp1 = grp1.rename({'Genes':'genesymbol', 'proteinlocation':'proteinlocation_new2'}, axis=1)
grp1

,genesymbol,proteinlocation_new2,Chrompos-Ref-Alt_new
0,ABCG2,p.Q141K,4-88131171-G-T
1,CACNA1S,p.R1086H,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010..."
2,CACNA1S,p.R174W,1-201091993-G-A
3,CFTR,p.A1067T,7-117611640-G-A
4,CFTR,p.A455E,7-117548795-C-A
...,...,...,...
734,TPMT,rs9333570,6-18133890-C-T
735,UGT1A1,p.G71R,"2-233760498-G-C,2-233760498-G-A"
736,UGT1A1,rs3064744,"2-233760234-ATATA-ATATATATATA,2-233760234-ATAT..."
737,UGT1A1,rs887829,2-233759924-C-T


In [51]:
data = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Haplotype_Genes_Allele_Master_File_05082023.xlsx')
data

,genesymbol,Haplotype star allele,Locatoins,REF,ALT,alleledefinitionID,locationID,variantallele,Name,chromosomelocation,genelocation,proteinlocation,dbsnpID
0,CACNA1S,Reference,1:201060815-201060815,G,T,777262,777261,C,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559
1,CACNA1S,c.3257G>A,1:201060815-201060815,C,G,777266,777261,T,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559
2,CACNA1S,Reference,1:201091993-201091993,G,T,777262,777260,G,c.520C>T,g.201091993G>A,g.25574C>T,p.R174W,rs772226819
3,CACNA1S,c.520C>T,1:201091993-201091993,C,T,777264,777260,A,c.520C>T,g.201091993G>A,g.25574C>T,p.R174W,rs772226819
4,DPYD,Reference,1:97078987-97078987,A,C,778327,778326,G,c.3067C>A,g.97078987G>T,g.847073C>A,p.P1023T,rs114096998
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012,CYP3A5,*1,7:99652771-99652771,AGCCAGGGAGTCCCTG,A,778218,948626,A,27126_27127insT,g.99652771dup,g.32228dup,p.T346fs,rs41303343
2013,CYP2D6,*1,22:42129084-42129084,C,T,777936,1103013,A,1708delT,g.42129084del,g.6727del,p.W152fs,rs5030655
2014,CYP2D6,*6,22:42129084-42129084,AAAG,A,777946,1103013,delA,1708delT,g.42129084del,g.6727del,p.W152fs,rs5030655
2015,G6PD,Mediterranean Haplotype,X:154532439-154532439,T,C,778984,778641,A,c.1311C>T,g.154532439G>A,g.20134C>T,p.Y437Y,rs2230037


In [52]:
data['proteinlocation_new'] = data['proteinlocation'].str.split(';').str[0]
data

,genesymbol,Haplotype star allele,Locatoins,REF,ALT,alleledefinitionID,locationID,variantallele,Name,chromosomelocation,genelocation,proteinlocation,dbsnpID,proteinlocation_new
0,CACNA1S,Reference,1:201060815-201060815,G,T,777262,777261,C,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,p.R1086H
1,CACNA1S,c.3257G>A,1:201060815-201060815,C,G,777266,777261,T,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,p.R1086H
2,CACNA1S,Reference,1:201091993-201091993,G,T,777262,777260,G,c.520C>T,g.201091993G>A,g.25574C>T,p.R174W,rs772226819,p.R174W
3,CACNA1S,c.520C>T,1:201091993-201091993,C,T,777264,777260,A,c.520C>T,g.201091993G>A,g.25574C>T,p.R174W,rs772226819,p.R174W
4,DPYD,Reference,1:97078987-97078987,A,C,778327,778326,G,c.3067C>A,g.97078987G>T,g.847073C>A,p.P1023T,rs114096998,p.P1023T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012,CYP3A5,*1,7:99652771-99652771,AGCCAGGGAGTCCCTG,A,778218,948626,A,27126_27127insT,g.99652771dup,g.32228dup,p.T346fs,rs41303343,p.T346fs
2013,CYP2D6,*1,22:42129084-42129084,C,T,777936,1103013,A,1708delT,g.42129084del,g.6727del,p.W152fs,rs5030655,p.W152fs
2014,CYP2D6,*6,22:42129084-42129084,AAAG,A,777946,1103013,delA,1708delT,g.42129084del,g.6727del,p.W152fs,rs5030655,p.W152fs
2015,G6PD,Mediterranean Haplotype,X:154532439-154532439,T,C,778984,778641,A,c.1311C>T,g.154532439G>A,g.20134C>T,p.Y437Y,rs2230037,p.Y437Y


In [54]:
merged = pd.merge(data, grp1, on = ['genesymbol', 'proteinlocation_new'], how = 'left', sort=False)
merged

,genesymbol,Haplotype star allele,Locatoins,REF,ALT,alleledefinitionID,locationID,variantallele,Name,chromosomelocation,genelocation,proteinlocation,dbsnpID,proteinlocation_new,Chrompos-Ref-Alt_new
0,CACNA1S,Reference,1:201060815-201060815,G,T,777262,777261,C,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,p.R1086H,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010..."
1,CACNA1S,c.3257G>A,1:201060815-201060815,C,G,777266,777261,T,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,p.R1086H,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010..."
2,CACNA1S,Reference,1:201091993-201091993,G,T,777262,777260,G,c.520C>T,g.201091993G>A,g.25574C>T,p.R174W,rs772226819,p.R174W,1-201091993-G-A
3,CACNA1S,c.520C>T,1:201091993-201091993,C,T,777264,777260,A,c.520C>T,g.201091993G>A,g.25574C>T,p.R174W,rs772226819,p.R174W,1-201091993-G-A
4,DPYD,Reference,1:97078987-97078987,A,C,778327,778326,G,c.3067C>A,g.97078987G>T,g.847073C>A,p.P1023T,rs114096998,p.P1023T,1-97078987-G-T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012,CYP3A5,*1,7:99652771-99652771,AGCCAGGGAGTCCCTG,A,778218,948626,A,27126_27127insT,g.99652771dup,g.32228dup,p.T346fs,rs41303343,p.T346fs,NaN
2013,CYP2D6,*1,22:42129084-42129084,C,T,777936,1103013,A,1708delT,g.42129084del,g.6727del,p.W152fs,rs5030655,p.W152fs,NaN
2014,CYP2D6,*6,22:42129084-42129084,AAAG,A,777946,1103013,delA,1708delT,g.42129084del,g.6727del,p.W152fs,rs5030655,p.W152fs,NaN
2015,G6PD,Mediterranean Haplotype,X:154532439-154532439,T,C,778984,778641,A,c.1311C>T,g.154532439G>A,g.20134C>T,p.Y437Y,rs2230037,p.Y437Y,NaN


In [56]:
merged['Chrompos-Ref-Alt_new'].value_counts()

22-42126611-C-G                    93
22-42127941-G-A                    71
10-94842866-A-G                    31
22-42130692-G-A                    26
12-21176804-A-G                    20
                                   ..
X-154532677-G-C                     1
X-154532676-C-T                     1
X-154532674-C-G,X-154532674-C-A     1
X-154532667-G-A                     1
22-42126747-G-A                     1
Name: Chrompos-Ref-Alt_new, Length: 682, dtype: int64

In [57]:
merged.to_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/mapped_variants.xlsx', index=False)

In [58]:
rsid = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/resulted_vr_rsids1b.txt', sep='\t')
rsid

,Uploaded_variant_hg38,Chrom-pos-Ref-Alt,rsID_hg38
0,rs55640102,"10-94852914-A-C,","rs55640102,"
1,rs55640102,"10-94852914-A-T,","rs55640102,"
2,rs375781227,"10-94781944-G-A,","rs375781227,"
3,rs375781227,"10-94781944-G-C,","rs375781227,"
4,rs12248560,"10-94761900-C-A,","rs12248560,"
...,...,...,...
57,rs761191455,13-48041104-G-GG,NaN
58,rs1457579126,13-48040981-AA-A,rs1457579126
59,rs759836180,"6-18149034-T-TT,",-
60,rs35350960,"2-233760973-C-A,","rs35350960, ,"


In [59]:
rsid['Chrompos-Ref-Alt_new'] = rsid['Chrom-pos-Ref-Alt'].str.split(',')
rsid = rsid.explode('Chrompos-Ref-Alt_new')
rsid

,Uploaded_variant_hg38,Chrom-pos-Ref-Alt,rsID_hg38,Chrompos-Ref-Alt_new
0,rs55640102,"10-94852914-A-C,","rs55640102,",10-94852914-A-C
0,rs55640102,"10-94852914-A-C,","rs55640102,",
1,rs55640102,"10-94852914-A-T,","rs55640102,",10-94852914-A-T
1,rs55640102,"10-94852914-A-T,","rs55640102,",
2,rs375781227,"10-94781944-G-A,","rs375781227,",10-94781944-G-A
...,...,...,...,...
59,rs759836180,"6-18149034-T-TT,",-,
60,rs35350960,"2-233760973-C-A,","rs35350960, ,",2-233760973-C-A
60,rs35350960,"2-233760973-C-A,","rs35350960, ,",
61,rs35350960,"2-233760973-C-T,","rs35350960, ,",2-233760973-C-T


In [60]:
rsid.to_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/rsid.xlsx', index=False)

In [61]:
rsid = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/rsid.xlsx')
rsid

,Uploaded_variant_hg38,Chrompos-Ref-Alt_new
0,rs55640102,10-94852914-A-C
1,rs55640102,10-94852914-A-T
2,rs375781227,10-94781944-G-A
3,rs375781227,10-94781944-G-C
4,rs12248560,10-94761900-C-A
...,...,...
57,rs761191455,13-48041104-G-GG
58,rs1457579126,13-48040981-AA-A
59,rs759836180,6-18149034-T-TT
60,rs35350960,2-233760973-C-A


In [64]:
grp2 = rsid.groupby(['Uploaded_variant_hg38']).agg({'Chrompos-Ref-Alt_new': lambda x: ','.join(str(i) for i in x.unique())}).reset_index()
grp2 = grp2.rename({'Uploaded_variant_hg38':'dbsnpID'}, axis=1)
grp2

,dbsnpID,Chrompos-Ref-Alt_new
0,rs12248560,"10-94761900-C-A,10-94761900-C-T"
1,rs1304490498,10-94942212-AAGAAATGGAA-A
2,rs137852337,"X-154532434-C-G,X-154532434-C-T"
3,rs1457579126,13-48040981-AA-A
4,rs1602566413,22-42126962-C-G
5,rs17376848,1-97450068-A-G
6,rs2230037,X-154532439-A-G
7,rs267608279,22-42127962-TG-T
8,rs28371706,"22-42129770-G-A,22-42129770-G-T"
9,rs35350960,"2-233760973-C-A,2-233760973-C-T"


In [65]:
merged_2 = pd.merge(merged, grp2, on = ['dbsnpID'], how = 'left', sort=False)
merged_2

,genesymbol,Haplotype star allele,Locatoins,REF,ALT,alleledefinitionID,locationID,variantallele,Name,chromosomelocation,genelocation,proteinlocation,dbsnpID,proteinlocation_new,Chrompos-Ref-Alt_new_x,Chrompos-Ref-Alt_new_y
0,CACNA1S,Reference,1:201060815-201060815,G,T,777262,777261,C,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,p.R1086H,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...",NaN
1,CACNA1S,c.3257G>A,1:201060815-201060815,C,G,777266,777261,T,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,p.R1086H,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...",NaN
2,CACNA1S,Reference,1:201091993-201091993,G,T,777262,777260,G,c.520C>T,g.201091993G>A,g.25574C>T,p.R174W,rs772226819,p.R174W,1-201091993-G-A,NaN
3,CACNA1S,c.520C>T,1:201091993-201091993,C,T,777264,777260,A,c.520C>T,g.201091993G>A,g.25574C>T,p.R174W,rs772226819,p.R174W,1-201091993-G-A,NaN
4,DPYD,Reference,1:97078987-97078987,A,C,778327,778326,G,c.3067C>A,g.97078987G>T,g.847073C>A,p.P1023T,rs114096998,p.P1023T,1-97078987-G-T,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012,CYP3A5,*1,7:99652771-99652771,AGCCAGGGAGTCCCTG,A,778218,948626,A,27126_27127insT,g.99652771dup,g.32228dup,p.T346fs,rs41303343,p.T346fs,NaN,7-99652771-A-AA
2013,CYP2D6,*1,22:42129084-42129084,C,T,777936,1103013,A,1708delT,g.42129084del,g.6727del,p.W152fs,rs5030655,p.W152fs,NaN,22-42129083-CA-C
2014,CYP2D6,*6,22:42129084-42129084,AAAG,A,777946,1103013,delA,1708delT,g.42129084del,g.6727del,p.W152fs,rs5030655,p.W152fs,NaN,22-42129083-CA-C
2015,G6PD,Mediterranean Haplotype,X:154532439-154532439,T,C,778984,778641,A,c.1311C>T,g.154532439G>A,g.20134C>T,p.Y437Y,rs2230037,p.Y437Y,NaN,X-154532439-A-G


In [66]:
merged_2.to_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/Shyam_final.xlsx', index=False)

In [67]:
merged_2 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/Shyam_final.xlsx')
merged_2

,genesymbol,Haplotype star allele,Locatoins,REF,ALT,alleledefinitionID,locationID,variantallele,Name,chromosomelocation,genelocation,proteinlocation,dbsnpID,proteinlocation_new,Chrompos-Ref-Alt_new_x,Chrompos-Ref-Alt_new_y,Final
0,CACNA1S,Reference,1:201060815-201060815,G,T,777262,777261,C,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,p.R1086H,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...",NaN,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010..."
1,CACNA1S,c.3257G>A,1:201060815-201060815,C,G,777266,777261,T,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,p.R1086H,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...",NaN,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010..."
2,CACNA1S,Reference,1:201091993-201091993,G,T,777262,777260,G,c.520C>T,g.201091993G>A,g.25574C>T,p.R174W,rs772226819,p.R174W,1-201091993-G-A,NaN,1-201091993-G-A
3,CACNA1S,c.520C>T,1:201091993-201091993,C,T,777264,777260,A,c.520C>T,g.201091993G>A,g.25574C>T,p.R174W,rs772226819,p.R174W,1-201091993-G-A,NaN,1-201091993-G-A
4,DPYD,Reference,1:97078987-97078987,A,C,778327,778326,G,c.3067C>A,g.97078987G>T,g.847073C>A,p.P1023T,rs114096998,p.P1023T,1-97078987-G-T,NaN,1-97078987-G-T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012,CYP3A5,*1,7:99652771-99652771,AGCCAGGGAGTCCCTG,A,778218,948626,A,27126_27127insT,g.99652771dup,g.32228dup,p.T346fs,rs41303343,p.T346fs,NaN,7-99652771-A-AA,7-99652771-A-AA
2013,CYP2D6,*1,22:42129084-42129084,C,T,777936,1103013,A,1708delT,g.42129084del,g.6727del,p.W152fs,rs5030655,p.W152fs,NaN,22-42129083-CA-C,22-42129083-CA-C
2014,CYP2D6,*6,22:42129084-42129084,AAAG,A,777946,1103013,delA,1708delT,g.42129084del,g.6727del,p.W152fs,rs5030655,p.W152fs,NaN,22-42129083-CA-C,22-42129083-CA-C
2015,G6PD,Mediterranean Haplotype,X:154532439-154532439,T,C,778984,778641,A,c.1311C>T,g.154532439G>A,g.20134C>T,p.Y437Y,rs2230037,p.Y437Y,NaN,X-154532439-A-G,X-154532439-A-G


In [68]:
merged_2['proteinlocation_new2'] = merged_2['proteinlocation'].str.split(';').str[1]
merged_2

,genesymbol,Haplotype star allele,Locatoins,REF,ALT,alleledefinitionID,locationID,variantallele,Name,chromosomelocation,genelocation,proteinlocation,dbsnpID,proteinlocation_new,Chrompos-Ref-Alt_new_x,Chrompos-Ref-Alt_new_y,Final,proteinlocation_new2
0,CACNA1S,Reference,1:201060815-201060815,G,T,777262,777261,C,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,p.R1086H,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...",NaN,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...",NaN
1,CACNA1S,c.3257G>A,1:201060815-201060815,C,G,777266,777261,T,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,p.R1086H,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...",NaN,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...",NaN
2,CACNA1S,Reference,1:201091993-201091993,G,T,777262,777260,G,c.520C>T,g.201091993G>A,g.25574C>T,p.R174W,rs772226819,p.R174W,1-201091993-G-A,NaN,1-201091993-G-A,NaN
3,CACNA1S,c.520C>T,1:201091993-201091993,C,T,777264,777260,A,c.520C>T,g.201091993G>A,g.25574C>T,p.R174W,rs772226819,p.R174W,1-201091993-G-A,NaN,1-201091993-G-A,NaN
4,DPYD,Reference,1:97078987-97078987,A,C,778327,778326,G,c.3067C>A,g.97078987G>T,g.847073C>A,p.P1023T,rs114096998,p.P1023T,1-97078987-G-T,NaN,1-97078987-G-T,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012,CYP3A5,*1,7:99652771-99652771,AGCCAGGGAGTCCCTG,A,778218,948626,A,27126_27127insT,g.99652771dup,g.32228dup,p.T346fs,rs41303343,p.T346fs,NaN,7-99652771-A-AA,7-99652771-A-AA,NaN
2013,CYP2D6,*1,22:42129084-42129084,C,T,777936,1103013,A,1708delT,g.42129084del,g.6727del,p.W152fs,rs5030655,p.W152fs,NaN,22-42129083-CA-C,22-42129083-CA-C,NaN
2014,CYP2D6,*6,22:42129084-42129084,AAAG,A,777946,1103013,delA,1708delT,g.42129084del,g.6727del,p.W152fs,rs5030655,p.W152fs,NaN,22-42129083-CA-C,22-42129083-CA-C,NaN
2015,G6PD,Mediterranean Haplotype,X:154532439-154532439,T,C,778984,778641,A,c.1311C>T,g.154532439G>A,g.20134C>T,p.Y437Y,rs2230037,p.Y437Y,NaN,X-154532439-A-G,X-154532439-A-G,NaN


In [72]:
merged_2.proteinlocation_new2.isnull().sum()

1933

In [73]:
2017-1933

84

In [74]:
print(set(merged_2.proteinlocation_new2))

{' p.174_175insFRP', ' p.E156V', ' p.V18_V19insGV', ' p.Y240S', 'N363K', 'p.R227L', ' p.R2508G', ' p.R125L', 'p.T279S', 'p.R459P', 'p.D282Y', 'p.P62H', ' p.V119L', 'p.G410A', 'p.R198P', ' p.G169X', 'p.L128P', ' p.R150L', 'p.R463C', ' p.T130M', ' p.R487C', ' p.R163P', nan, ' p.T107I', ' p.T2206M'}


In [75]:
merged_2.to_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/shyam_second.xlsx', index=False)

In [76]:
merged_2 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/shyam_second.xlsx')
merged_2

,genesymbol,Haplotype star allele,Locatoins,REF,ALT,alleledefinitionID,locationID,variantallele,Name,chromosomelocation,genelocation,proteinlocation,dbsnpID,proteinlocation_new,Chrompos-Ref-Alt_new_x,Chrompos-Ref-Alt_new_y,Final,proteinlocation_new2
0,CACNA1S,Reference,1:201060815-201060815,G,T,777262,777261,C,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,p.R1086H,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...",NaN,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...",NaN
1,CACNA1S,c.3257G>A,1:201060815-201060815,C,G,777266,777261,T,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,p.R1086H,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...",NaN,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...",NaN
2,CACNA1S,Reference,1:201091993-201091993,G,T,777262,777260,G,c.520C>T,g.201091993G>A,g.25574C>T,p.R174W,rs772226819,p.R174W,1-201091993-G-A,NaN,1-201091993-G-A,NaN
3,CACNA1S,c.520C>T,1:201091993-201091993,C,T,777264,777260,A,c.520C>T,g.201091993G>A,g.25574C>T,p.R174W,rs772226819,p.R174W,1-201091993-G-A,NaN,1-201091993-G-A,NaN
4,DPYD,Reference,1:97078987-97078987,A,C,778327,778326,G,c.3067C>A,g.97078987G>T,g.847073C>A,p.P1023T,rs114096998,p.P1023T,1-97078987-G-T,NaN,1-97078987-G-T,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012,CYP3A5,*1,7:99652771-99652771,AGCCAGGGAGTCCCTG,A,778218,948626,A,27126_27127insT,g.99652771dup,g.32228dup,p.T346fs,rs41303343,p.T346fs,NaN,7-99652771-A-AA,7-99652771-A-AA,NaN
2013,CYP2D6,*1,22:42129084-42129084,C,T,777936,1103013,A,1708delT,g.42129084del,g.6727del,p.W152fs,rs5030655,p.W152fs,NaN,22-42129083-CA-C,22-42129083-CA-C,NaN
2014,CYP2D6,*6,22:42129084-42129084,AAAG,A,777946,1103013,delA,1708delT,g.42129084del,g.6727del,p.W152fs,rs5030655,p.W152fs,NaN,22-42129083-CA-C,22-42129083-CA-C,NaN
2015,G6PD,Mediterranean Haplotype,X:154532439-154532439,T,C,778984,778641,A,c.1311C>T,g.154532439G>A,g.20134C>T,p.Y437Y,rs2230037,p.Y437Y,NaN,X-154532439-A-G,X-154532439-A-G,NaN


In [78]:
merged_final = pd.merge(merged_2, grp1, on = ['genesymbol', 'proteinlocation_new2'], how = 'left', sort=False)
merged_final

,genesymbol,Haplotype star allele,Locatoins,REF,ALT,alleledefinitionID,locationID,variantallele,Name,chromosomelocation,genelocation,proteinlocation,dbsnpID,proteinlocation_new,Chrompos-Ref-Alt_new_x,Chrompos-Ref-Alt_new_y,Final,proteinlocation_new2,Chrompos-Ref-Alt_new
0,CACNA1S,Reference,1:201060815-201060815,G,T,777262,777261,C,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,p.R1086H,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...",NaN,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...",NaN,NaN
1,CACNA1S,c.3257G>A,1:201060815-201060815,C,G,777266,777261,T,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,p.R1086H,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...",NaN,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...",NaN,NaN
2,CACNA1S,Reference,1:201091993-201091993,G,T,777262,777260,G,c.520C>T,g.201091993G>A,g.25574C>T,p.R174W,rs772226819,p.R174W,1-201091993-G-A,NaN,1-201091993-G-A,NaN,NaN
3,CACNA1S,c.520C>T,1:201091993-201091993,C,T,777264,777260,A,c.520C>T,g.201091993G>A,g.25574C>T,p.R174W,rs772226819,p.R174W,1-201091993-G-A,NaN,1-201091993-G-A,NaN,NaN
4,DPYD,Reference,1:97078987-97078987,A,C,778327,778326,G,c.3067C>A,g.97078987G>T,g.847073C>A,p.P1023T,rs114096998,p.P1023T,1-97078987-G-T,NaN,1-97078987-G-T,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012,CYP3A5,*1,7:99652771-99652771,AGCCAGGGAGTCCCTG,A,778218,948626,A,27126_27127insT,g.99652771dup,g.32228dup,p.T346fs,rs41303343,p.T346fs,NaN,7-99652771-A-AA,7-99652771-A-AA,NaN,NaN
2013,CYP2D6,*1,22:42129084-42129084,C,T,777936,1103013,A,1708delT,g.42129084del,g.6727del,p.W152fs,rs5030655,p.W152fs,NaN,22-42129083-CA-C,22-42129083-CA-C,NaN,NaN
2014,CYP2D6,*6,22:42129084-42129084,AAAG,A,777946,1103013,delA,1708delT,g.42129084del,g.6727del,p.W152fs,rs5030655,p.W152fs,NaN,22-42129083-CA-C,22-42129083-CA-C,NaN,NaN
2015,G6PD,Mediterranean Haplotype,X:154532439-154532439,T,C,778984,778641,A,c.1311C>T,g.154532439G>A,g.20134C>T,p.Y437Y,rs2230037,p.Y437Y,NaN,X-154532439-A-G,X-154532439-A-G,NaN,NaN


In [80]:
merged_final.to_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/madhu_final_file.xlsx', index=False)

In [82]:
grp1[grp1['proteinlocation_new2'] == 'p.R150H']

,genesymbol,proteinlocation_new2,Chrompos-Ref-Alt_new
105,CYP2C19,p.R150H,10-94775507-G-A
185,CYP2C9,p.R150H,10-94942309-G-A


In [85]:
shyam = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/shyam_data_mapped_new.xlsx')
shyam

,genesymbol,Haplotype star allele,Locatoins,REF,ALT,alleledefinitionID,locationID,variantallele,Name,chromosomelocation,genelocation,proteinlocation,dbsnpID,Final_2
0,CACNA1S,Reference,1:201060815-201060815,G,T,777262,777261,C,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010..."
1,CACNA1S,c.3257G>A,1:201060815-201060815,C,G,777266,777261,T,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010..."
2,CACNA1S,Reference,1:201091993-201091993,G,T,777262,777260,G,c.520C>T,g.201091993G>A,g.25574C>T,p.R174W,rs772226819,"1-201091993-G-A,"
3,CACNA1S,c.520C>T,1:201091993-201091993,C,T,777264,777260,A,c.520C>T,g.201091993G>A,g.25574C>T,p.R174W,rs772226819,"1-201091993-G-A,"
4,DPYD,Reference,1:97078987-97078987,A,C,778327,778326,G,c.3067C>A,g.97078987G>T,g.847073C>A,p.P1023T,rs114096998,"1-97078987-G-T,"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012,CYP3A5,*1,7:99652771-99652771,AGCCAGGGAGTCCCTG,A,778218,948626,A,27126_27127insT,g.99652771dup,g.32228dup,p.T346fs,rs41303343,"7-99652771-A-AA,"
2013,CYP2D6,*1,22:42129084-42129084,C,T,777936,1103013,A,1708delT,g.42129084del,g.6727del,p.W152fs,rs5030655,"22-42129083-CA-C,"
2014,CYP2D6,*6,22:42129084-42129084,AAAG,A,777946,1103013,delA,1708delT,g.42129084del,g.6727del,p.W152fs,rs5030655,"22-42129083-CA-C,"
2015,G6PD,Mediterranean Haplotype,X:154532439-154532439,T,C,778984,778641,A,c.1311C>T,g.154532439G>A,g.20134C>T,p.Y437Y,rs2230037,"X-154532439-A-G,"


In [86]:
shyam['Chrom_pos'] = shyam['Final_2'].str.rstrip(',')
shyam

,genesymbol,Haplotype star allele,Locatoins,REF,ALT,alleledefinitionID,locationID,variantallele,Name,chromosomelocation,genelocation,proteinlocation,dbsnpID,Final_2,Chrom_pos
0,CACNA1S,Reference,1:201060815-201060815,G,T,777262,777261,C,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...","1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010..."
1,CACNA1S,c.3257G>A,1:201060815-201060815,C,G,777266,777261,T,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...","1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010..."
2,CACNA1S,Reference,1:201091993-201091993,G,T,777262,777260,G,c.520C>T,g.201091993G>A,g.25574C>T,p.R174W,rs772226819,"1-201091993-G-A,",1-201091993-G-A
3,CACNA1S,c.520C>T,1:201091993-201091993,C,T,777264,777260,A,c.520C>T,g.201091993G>A,g.25574C>T,p.R174W,rs772226819,"1-201091993-G-A,",1-201091993-G-A
4,DPYD,Reference,1:97078987-97078987,A,C,778327,778326,G,c.3067C>A,g.97078987G>T,g.847073C>A,p.P1023T,rs114096998,"1-97078987-G-T,",1-97078987-G-T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012,CYP3A5,*1,7:99652771-99652771,AGCCAGGGAGTCCCTG,A,778218,948626,A,27126_27127insT,g.99652771dup,g.32228dup,p.T346fs,rs41303343,"7-99652771-A-AA,",7-99652771-A-AA
2013,CYP2D6,*1,22:42129084-42129084,C,T,777936,1103013,A,1708delT,g.42129084del,g.6727del,p.W152fs,rs5030655,"22-42129083-CA-C,",22-42129083-CA-C
2014,CYP2D6,*6,22:42129084-42129084,AAAG,A,777946,1103013,delA,1708delT,g.42129084del,g.6727del,p.W152fs,rs5030655,"22-42129083-CA-C,",22-42129083-CA-C
2015,G6PD,Mediterranean Haplotype,X:154532439-154532439,T,C,778984,778641,A,c.1311C>T,g.154532439G>A,g.20134C>T,p.Y437Y,rs2230037,"X-154532439-A-G,",X-154532439-A-G


In [88]:
shyam.Final_2.iloc[0]

'1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-201060815-C-T,'

In [89]:
shyam['chrom_pos'] = shyam['Chrom_pos'].str.split(',')
shyam = shyam.explode('chrom_pos')
shyam

,genesymbol,Haplotype star allele,Locatoins,REF,ALT,alleledefinitionID,locationID,variantallele,Name,chromosomelocation,genelocation,proteinlocation,dbsnpID,Final_2,Chrom_pos,chrom_pos
0,CACNA1S,Reference,1:201060815-201060815,G,T,777262,777261,C,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...","1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...",1-201061290-TCT-ATG
0,CACNA1S,Reference,1:201060815-201060815,G,T,777262,777261,C,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...","1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...",1-201061290-TCT-GTG
0,CACNA1S,Reference,1:201060815-201060815,G,T,777262,777261,C,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...","1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...",1-201060815-C-T
1,CACNA1S,c.3257G>A,1:201060815-201060815,C,G,777266,777261,T,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...","1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...",1-201061290-TCT-ATG
1,CACNA1S,c.3257G>A,1:201060815-201060815,C,G,777266,777261,T,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...","1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...",1-201061290-TCT-GTG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012,CYP3A5,*1,7:99652771-99652771,AGCCAGGGAGTCCCTG,A,778218,948626,A,27126_27127insT,g.99652771dup,g.32228dup,p.T346fs,rs41303343,"7-99652771-A-AA,",7-99652771-A-AA,7-99652771-A-AA
2013,CYP2D6,*1,22:42129084-42129084,C,T,777936,1103013,A,1708delT,g.42129084del,g.6727del,p.W152fs,rs5030655,"22-42129083-CA-C,",22-42129083-CA-C,22-42129083-CA-C
2014,CYP2D6,*6,22:42129084-42129084,AAAG,A,777946,1103013,delA,1708delT,g.42129084del,g.6727del,p.W152fs,rs5030655,"22-42129083-CA-C,",22-42129083-CA-C,22-42129083-CA-C
2015,G6PD,Mediterranean Haplotype,X:154532439-154532439,T,C,778984,778641,A,c.1311C>T,g.154532439G>A,g.20134C>T,p.Y437Y,rs2230037,"X-154532439-A-G,",X-154532439-A-G,X-154532439-A-G


In [99]:
shyam.to_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/mapped_data_main_shyam.xlsx', index=False)

In [98]:
shyam['chr_new'] = shyam['Locatoins'].str.split(':').str[0]
shyam['location_new'] = shyam['Locatoins'].str.split('-').str[-1]
shyam

,genesymbol,Haplotype star allele,Locatoins,REF,ALT,alleledefinitionID,locationID,variantallele,Name,chromosomelocation,genelocation,proteinlocation,dbsnpID,Final_2,Chrom_pos,chrom_pos,chr_new,location_new
0,CACNA1S,Reference,1:201060815-201060815,G,T,777262,777261,C,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...","1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...",1-201061290-TCT-ATG,1,201060815
0,CACNA1S,Reference,1:201060815-201060815,G,T,777262,777261,C,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...","1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...",1-201061290-TCT-GTG,1,201060815
0,CACNA1S,Reference,1:201060815-201060815,G,T,777262,777261,C,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...","1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...",1-201060815-C-T,1,201060815
1,CACNA1S,c.3257G>A,1:201060815-201060815,C,G,777266,777261,T,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...","1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...",1-201061290-TCT-ATG,1,201060815
1,CACNA1S,c.3257G>A,1:201060815-201060815,C,G,777266,777261,T,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,"1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...","1-201061290-TCT-ATG,1-201061290-TCT-GTG,1-2010...",1-201061290-TCT-GTG,1,201060815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012,CYP3A5,*1,7:99652771-99652771,AGCCAGGGAGTCCCTG,A,778218,948626,A,27126_27127insT,g.99652771dup,g.32228dup,p.T346fs,rs41303343,"7-99652771-A-AA,",7-99652771-A-AA,7-99652771-A-AA,7,99652771
2013,CYP2D6,*1,22:42129084-42129084,C,T,777936,1103013,A,1708delT,g.42129084del,g.6727del,p.W152fs,rs5030655,"22-42129083-CA-C,",22-42129083-CA-C,22-42129083-CA-C,22,42129084
2014,CYP2D6,*6,22:42129084-42129084,AAAG,A,777946,1103013,delA,1708delT,g.42129084del,g.6727del,p.W152fs,rs5030655,"22-42129083-CA-C,",22-42129083-CA-C,22-42129083-CA-C,22,42129084
2015,G6PD,Mediterranean Haplotype,X:154532439-154532439,T,C,778984,778641,A,c.1311C>T,g.154532439G>A,g.20134C>T,p.Y437Y,rs2230037,"X-154532439-A-G,",X-154532439-A-G,X-154532439-A-G,X,154532439


In [100]:
dt_new = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/mapped_data_main_shyam.xlsx')
dt_new

,genesymbol,Haplotype star allele,Locatoins,REF,ALT,Locations_new,alleledefinitionID,locationID,variantallele,Name,chromosomelocation,genelocation,proteinlocation,dbsnpID,chrom_pos
0,CACNA1S,Reference,1:201060815-201060815,G,T,1-201060815-G-T,777262,777261,C,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,1-201061290-TCT-ATG
1,CACNA1S,Reference,1:201060815-201060815,G,T,1-201060815-G-T,777262,777261,C,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,1-201061290-TCT-GTG
2,CACNA1S,Reference,1:201060815-201060815,G,T,1-201060815-G-T,777262,777261,C,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,1-201060815-C-T
3,CACNA1S,c.3257G>A,1:201060815-201060815,C,G,1-201060815-C-G,777266,777261,T,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,1-201061290-TCT-ATG
4,CACNA1S,c.3257G>A,1:201060815-201060815,C,G,1-201060815-C-G,777266,777261,T,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,1-201061290-TCT-GTG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2926,CYP3A5,*1,7:99652771-99652771,AGCCAGGGAGTCCCTG,A,7-99652771-AGCCAGGGAGTCCCTG-A,778218,948626,A,27126_27127insT,g.99652771dup,g.32228dup,p.T346fs,rs41303343,7-99652771-A-AA
2927,CYP2D6,*1,22:42129084-42129084,C,T,22-42129084-C-T,777936,1103013,A,1708delT,g.42129084del,g.6727del,p.W152fs,rs5030655,22-42129083-CA-C
2928,CYP2D6,*6,22:42129084-42129084,AAAG,A,22-42129084-AAAG-A,777946,1103013,delA,1708delT,g.42129084del,g.6727del,p.W152fs,rs5030655,22-42129083-CA-C
2929,G6PD,Mediterranean Haplotype,X:154532439-154532439,T,C,X-154532439-T-C,778984,778641,A,c.1311C>T,g.154532439G>A,g.20134C>T,p.Y437Y,rs2230037,X-154532439-A-G


In [107]:
dt_madhu = dt_new.copy()
dt_madhu['Locations_new'] = dt_madhu['Locations_new'].str.split('-').str[0:2]
dt_madhu['new_chrom_pos'] = dt_madhu['chrom_pos'].str.split('-').str[0:2]
dt_madhu

,genesymbol,Haplotype star allele,Locatoins,REF,ALT,Locations_new,alleledefinitionID,locationID,variantallele,Name,chromosomelocation,genelocation,proteinlocation,dbsnpID,chrom_pos,new_chrom_pos
0,CACNA1S,Reference,1:201060815-201060815,G,T,"[1, 201060815]",777262,777261,C,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,1-201061290-TCT-ATG,"[1, 201061290]"
1,CACNA1S,Reference,1:201060815-201060815,G,T,"[1, 201060815]",777262,777261,C,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,1-201061290-TCT-GTG,"[1, 201061290]"
2,CACNA1S,Reference,1:201060815-201060815,G,T,"[1, 201060815]",777262,777261,C,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,1-201060815-C-T,"[1, 201060815]"
3,CACNA1S,c.3257G>A,1:201060815-201060815,C,G,"[1, 201060815]",777266,777261,T,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,1-201061290-TCT-ATG,"[1, 201061290]"
4,CACNA1S,c.3257G>A,1:201060815-201060815,C,G,"[1, 201060815]",777266,777261,T,c.3257G>A,g.201060815C>T,g.56752G>A,p.R1086H,rs1800559,1-201061290-TCT-GTG,"[1, 201061290]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2926,CYP3A5,*1,7:99652771-99652771,AGCCAGGGAGTCCCTG,A,"[7, 99652771]",778218,948626,A,27126_27127insT,g.99652771dup,g.32228dup,p.T346fs,rs41303343,7-99652771-A-AA,"[7, 99652771]"
2927,CYP2D6,*1,22:42129084-42129084,C,T,"[22, 42129084]",777936,1103013,A,1708delT,g.42129084del,g.6727del,p.W152fs,rs5030655,22-42129083-CA-C,"[22, 42129083]"
2928,CYP2D6,*6,22:42129084-42129084,AAAG,A,"[22, 42129084]",777946,1103013,delA,1708delT,g.42129084del,g.6727del,p.W152fs,rs5030655,22-42129083-CA-C,"[22, 42129083]"
2929,G6PD,Mediterranean Haplotype,X:154532439-154532439,T,C,"[X, 154532439]",778984,778641,A,c.1311C>T,g.154532439G>A,g.20134C>T,p.Y437Y,rs2230037,X-154532439-A-G,"[X, 154532439]"


In [103]:
matched_data = dt_new[dt_new['Locations_new'] == dt_new['chrom_pos']].copy()
matched_data

,genesymbol,Haplotype star allele,Locatoins,REF,ALT,Locations_new,alleledefinitionID,locationID,variantallele,Name,chromosomelocation,genelocation,proteinlocation,dbsnpID,chrom_pos
18,DPYD,Reference,1:97079076-97079076,A,C,1-97079076-A-C,778327,778321,A,c.2978T>G,g.97079076A>C,g.846984T>G,p.L993R,rs139459586,1-97079076-A-C
25,DPYD,c.2915A>G,1:97079139-97079139,T,C,1-97079139-T-C,778475,778318,C,c.2915A>G,g.97079139T>C,g.846921A>G,p.Q972R,rs145529148,1-97079139-T-C
39,DPYD,c.2582A>G,1:97193109-97193109,T,C,1-97193109-T-C,778461,778311,C,c.2582A>G,g.97193109T>C,g.732951A>G,p.K861R,rs60139309,1-97193109-T-C
47,DPYD,c.2279C>T,1:97305279-97305279,G,A,1-97305279-G-A,778453,778307,A,c.2279C>T,g.97305279G>A,g.620781C>T,p.T760I,rs112766203,1-97305279-G-A
48,DPYD,Reference,1:97305363-97305363,A,C,1-97305363-A-C,778327,778306,A,c.2195T>G,g.97305363A>C,g.620697T>G,p.V732G,rs60511679,1-97305363-A-C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2752,G6PD,Shenzen,X:154535180-154535180,C,T,X-154535180-C-T,778772,778537,T,c.473G>A,g.154535180C>T,g.17393G>A,p.C158Y,rs782487723,X-154535180-C-T
2786,DPYD,c.1896T>C,1:97450068-97450068,A,G,1-97450068-A-G,778431,778296,G,c.1896T>C,g.97450068A>G,g.475992T>C,p.F632F,rs17376848,1-97450068-A-G
2794,CYP2D6,*1,22:42129033-42129033,C,T,22-42129033-C-T,777936,777853,C,1759G>A; 1759G>T,g.42129033C>T; g.42129033C>A,g.6778G>A; g.6778G>T,p.G169R; p.G169X,rs5030865,22-42129033-C-T
2799,CYP2D6,*114,22:42129033-42129033,C,A,22-42129033-C-A,778146,777853,T,1759G>A; 1759G>T,g.42129033C>T; g.42129033C>A,g.6778G>A; g.6778G>T,p.G169R; p.G169X,rs5030865,22-42129033-C-A
